In [ ]:
# Install latest version sklearn (0.23)
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


In [ ]:
# Imports
import pandas as pd
import numpy as np
import os
import shutil
from collections import Counter
import matplotlib.pyplot as plt
import time
import gc
import sys
import random
import pickle

from scipy.stats import iqr
from sklearn import preprocessing

import datetime
import logging

# Deep learning
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

from IPython.display import display
# Set pandas options
pd.options.display.max_rows = 2000

In [ ]:
# Global variables 
THR_E = 100 # interaction threshold of exercises (E) for user
BATCH_SIZE = 128

AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 200
N_SELECT_PER_EPOCH = 100000 # Random select N samples for each epoch from train/val set
VAL_EVERY_N_EPOCHS = 5
PRINT_EVERY_N_BATCHES = 50 

SAVE_DICTS = True

In [ ]:
# Experiment date and number!
date = datetime.datetime.today().strftime("%d-%b")
experiment = "-riiid-2"
OUTPUT_FOLDER = date + experiment 
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Selected features
selections = ["E", "r", "lt"]
assert((selections[0] == "E") & (selections[1]=="r"))

In [ ]:
# Choose model setting
ENC_EMB, ENC_DENSE = [0], [] 
DEC_EMB, DEC_DENSE = [1], [2]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDER_FEATHER = "/content/drive/My Drive/kaggle-riiid/feather-files"
PREPROCESS_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-preprocessing.ipynb"
MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/demo-riiid-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook
# MODEL_FILE = "/content/drive/My\ Drive/Colab\ Notebooks/riiid-functional-transformer.ipynb" # https://stackoverflow.com/questions/57464810/how-to-run-a-jupyter-notebook-with-space-in-relative-path-from-another-notebook


Mounted at /content/drive


In [ ]:
# Add functions to preprocess data
%run $PREPROCESS_FILE

In [ ]:
# Create logger
logging = create_logging(OUTPUT_FOLDER)

In [ ]:
%%time
# Read all dataframes from feather files and print out
train = read_df_print(os.path.join(FOLDER_FEATHER, "train.feather")) 
questions = read_df_print(os.path.join(FOLDER_FEATHER, "questions.feather"))
lectures = read_df_print(os.path.join(FOLDER_FEATHER, "lectures.feather"))

(101230332, 10)


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,None
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False


(13523, 5)


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92


(418, 4)


,lecture_id,tag,part,type_of
0,89,24584,5,concept
1,100,22243,1,concept
2,185,7035,6,concept


CPU times: user 1.69 s, sys: 1.89 s, total: 3.58 s
Wall time: 2.58 s


In [ ]:
# We don't need the lectures for now!
train = train.loc[train["content_type_id"]==0]

19-12 17:53 numexpr.utils INFO     NumExpr defaulting to 4 threads.


In [ ]:
# Save space by deleting some columns
del train["row_id"], train["user_answer"]

In [ ]:
N_questions, N_parts = questions["question_id"].nunique(), questions["part"].nunique()
N_response, N_task, N_lag, N_et = 2, 2, 1440, 301

In [ ]:
# %%time 
# # Split users into multiple parts based on THR_E
# train = split_into_more_users(train, ascending=USER_ASCENDING)
# print(f"Number of new_user_ids: {train.new_user_id.nunique()}")

In [ ]:
# Seq len of users [sorted same as df]. USED FOR TRAINING
seq_len_dict = dict(train["user_id"].value_counts())
users = train["user_id"].unique()
seq_lens = [seq_len_dict[user] for user in users]

In [ ]:
# Train/valid split
train_len = int(len(users)*0.9)
val_len = len(users) - train_len
train_users, val_users = users[:train_len], users[train_len:]

# PROBS of sequence
seq_len_train, seq_len_val = seq_lens[:train_len], seq_lens[train_len:]
PROBS_TRAIN = seq_len_train/np.sum(seq_len_train)
PROBS_VAL = seq_len_val/np.sum(seq_len_val)

In [ ]:
len(users)

393656

## Add all inputs/outputs

In [ ]:
p_lists, lt_lists, et_lists, tag_lists, task_lists, quantile_transformer_et, lt_cat, quantile_transformer_lt, et_cat, ltc, etc = [], [], [], [], [], [], [], [], [], [],[]

In [ ]:
# Add exercises
%time E_lists = return_E()

CPU times: user 24.7 s, sys: 1.31 s, total: 26 s
Wall time: 26 s


In [ ]:
# Add results, with or without start token
%time r_lists = return_r(add_start_token=False)

# delete to make space
del train["answered_correctly"]

CPU times: user 23 s, sys: 83.3 ms, total: 23.1 s
Wall time: 23.1 s


In [ ]:
if "ltc" in selections:
    %time ltc = return_ltc()

In [ ]:
if "lt" in selections:
    %time lt_lists, quantile_transformer_lt = return_lt()

CPU times: user 48 s, sys: 2.57 s, total: 50.5 s
Wall time: 50.1 s


In [ ]:
if "etc" in selections:
    %time etc = return_etc()

In [ ]:
if "p" in selections:
    %time p_lists = return_p()

In [ ]:
if "et" in selections:
    %time et_lists, quantile_transformer_et = return_et()

In [ ]:
if "tag" in selections:
    %time tag_lists = return_N_highest_tags() # TODO: specify N tags

In [ ]:
if "task" in selections:
    %time task_lists = return_task_binary()

In [ ]:
feature_mapping = {"E": E_lists, "r": r_lists, "p": p_lists, "et": et_lists, "lt": lt_lists, "tag": tag_lists, "task": task_lists, "et_std": et_lists, "ltc": ltc, "etc": etc}
type_mapping = {"E": tf.int32, "r": tf.int32, "p": tf.int32, "et": tf.float32, "et_std": tf.float32, "lt": tf.float32, "tag": tf.float32, "task": tf.float32, "ltc":tf.int32, "etc": tf.int32}
pad_mapping = {"E": N_questions, "r": N_response, "p": 0, "et": 0.0, "et_std": 0.0, "lt": 1.0, "tag": 2.0, "task": float(N_task), "ltc": N_lag+1, "etc": N_et+1}
vocab_mapping = {"E": N_questions+1, "r": N_response+2, "et":None, "p": N_parts+1, "lt": None, "tag": 3.0, "task": float(N_task+1), "et_std": float(300), "ltc": N_lag+2, "etc": N_et+2}
pad_shapes = {"E": [THR_E], "r": [THR_E+1], "p": [THR_E], "et": [THR_E], "et_std": [THR_E],  "lt": [THR_E], "tag": [THR_E], "task": [THR_E], "ltc": [THR_E], "etc": [THR_E]}

In [ ]:
def save_as_dict(df, filename):
    seq_dict = df.to_dict()
    with open(os.path.join(OUTPUT_FOLDER_DICT, filename), 'wb') as handle:
      pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
if SAVE_DICTS:
  OUTPUT_FOLDER_DICT = f"seq-dict-{THR_E}"
  os.makedirs(OUTPUT_FOLDER_DICT, exist_ok=True)

  for feature in selections:
      feat_df = feature_mapping[feature]
      feat_df = feat_df.apply(lambda x: x[-THR_E:]) # Take last THR_E in history 
      filename = f"{feature}.pickle"
      save_as_dict(feat_df, filename)
  shutil.move(OUTPUT_FOLDER_DICT, OUTPUT_FOLDER)

In [ ]:
# Add start token!
r_lists = r_lists.apply(lambda x: [3] + x) # Start token = 3
feature_mapping["r"] = r_lists

In [ ]:
vocab_sizes = [vocab_mapping[select] for select in selections]
feature_lists = [feature_mapping[select] for select in selections]

In [ ]:
PADDING_VALUES = tuple((pad_mapping[select] for select in selections))
OUTPUT_TYPES = tuple((type_mapping[select] for select in selections))
PADDED_SHAPES = tuple((pad_shapes[select] for select in selections))

In [ ]:
feature_lists

[user_id
 115           [5692, 5716, 128, 7860, 7922, 156, 51, 50, 789...
 124           [7900, 7876, 175, 1278, 2064, 2063, 2065, 3364...
 2746          [5273, 758, 5976, 236, 404, 382, 405, 873, 531...
 5382          [5000, 3944, 217, 5844, 5965, 4990, 5235, 6050...
 8623          [3915, 4750, 6456, 3968, 6104, 5738, 6435, 549...
                                     ...                        
 2147470770    [7900, 7876, 175, 1278, 2064, 2065, 2063, 3363...
 2147470777    [7900, 7876, 175, 1278, 2065, 2064, 2063, 3365...
 2147481750    [4137, 1270, 9261, 8201, 367, 378, 214, 6071, ...
 2147482216    [3748, 4765, 5474, 9261, 4665, 5987, 6666, 561...
 2147482888    [6147, 4792, 5738, 6102, 4748, 7956, 6435, 928...
 Name: content_id, Length: 393656, dtype: object, user_id
 115           [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, ...
 124           [3, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...
 2746          [3, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...
 5382          [3, 1, 0

In [ ]:
try:
  del train
except Exception as e:
  print("train already deleted")
gc.collect(), gc.collect()

(60, 0)

# Create train/val datasets

In [ ]:
# Lists of numpy arrays
train_list = [features.values[:train_len] for features in feature_lists]
val_list = [features.values[train_len:] for features in feature_lists]

In [ ]:
def select_window_size(*x):
    series_len = len(x[0]) # Length of series
    if series_len <= THR_E: # Just return the sequence!
        return x
    else: # Random select from sequence
        max_select = series_len - THR_E
        random_select = tf.random.uniform(shape=(), minval=0, maxval=max_select, dtype=tf.int32)
        x = [i[random_select:random_select+THR_E] for i in x]
        x = tuple(x)
        return x

In [ ]:
train_list

[array([list([5692, 5716, 128, 7860, 7922, 156, 51, 50, 7896, 7863, 152, 104, 108, 7900, 7901, 7971, 25, 183, 7926, 7927, 4, 7984, 45, 185, 55, 7876, 6, 172, 7898, 175, 100, 7859, 57, 7948, 151, 167, 7897, 7882, 7962, 1278, 2065, 2064, 2063, 3363, 3365, 3364]),
        list([7900, 7876, 175, 1278, 2064, 2063, 2065, 3364, 3365, 3363, 2948, 2947, 2946, 2595, 2593, 2594, 4492, 4120, 4696, 6116, 6173, 6370, 6909, 6910, 6908, 6911, 7218, 7216, 7217, 7219]),
        list([5273, 758, 5976, 236, 404, 382, 405, 873, 531, 775, 294, 714, 297, 297, 775, 1295, 10684, 1014, 484]),
        ...,
        list([4500, 4264, 8201, 214, 378, 3674, 4908, 5387, 4402, 6442, 4501, 3957, 4428, 6094, 4416, 4491, 6443, 6292, 5216, 3878, 5670, 6246, 6130, 6767, 6768, 6765, 6766, 6760, 6757, 6758, 6759, 10071, 10072, 10070, 10073, 6869, 6870, 6871, 6872, 6866, 6867, 6865, 6868, 10080, 10078, 10081, 10079, 6624, 6622, 6623, 6621, 6770, 6769, 6771, 6772]),
        list([7900, 7876, 175, 1278, 2065, 2064, 2063, 3364, 

In [ ]:
def create_train_dataset(N_training_per_epoch, probs_train):
  index = np.random.choice(train_len, N_training_per_epoch, replace=True, p = probs_train) # random indexing (similar to shuffling)
  tr_list = [features[index] for features in train_list] # Selecting by index 
  train_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*tr_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .prefetch(AUTO)
  )
  return train_ds

In [ ]:
def create_val_dataset(N_valid_per_epoch, probs_val):
  index = np.random.choice(val_len, N_valid_per_epoch, replace=True, p = probs_val) # random indexing (similar to shuffling)
  vl_list = [features[index] for features in val_list] # Selecting by index 
  val_ds = (tf.data.Dataset
                  .from_generator(lambda: iter(zip(*vl_list)), output_types=OUTPUT_TYPES)
                  .map(select_window_size)
                  .padded_batch(batch_size = BATCH_SIZE, padded_shapes = PADDED_SHAPES, padding_values = PADDING_VALUES)
                  .cache()
                  .prefetch(AUTO)
  )
  return val_ds

In [ ]:
val_dataset = create_val_dataset(N_SELECT_PER_EPOCH, PROBS_VAL)
a = iter(val_dataset) # Small check

In [ ]:
next(a)

(<tf.Tensor: shape=(128, 100), dtype=int32, numpy=
 array([[ 9861,  4852,  4125, ...,  1034,   931,  1142],
        [ 7900,  7876,   175, ..., 13523, 13523, 13523],
        [ 5592,  8457,  4728, ...,  3968,  9939,  4248],
        ...,
        [  677,  1280,   446, ...,  1432,  1431,  2172],
        [ 6814,  6813,  6816, ...,  3354,  3355,  3356],
        [ 1506,  1505,  2541, ...,   237,   721,   650]], dtype=int32)>,
 <tf.Tensor: shape=(128, 101), dtype=int32, numpy=
 array([[1, 0, 1, ..., 1, 1, 2],
        [3, 1, 1, ..., 2, 2, 2],
        [1, 1, 1, ..., 0, 0, 2],
        ...,
        [1, 1, 1, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 1, 2],
        [1, 1, 1, ..., 1, 1, 2]], dtype=int32)>,
 <tf.Tensor: shape=(128, 100), dtype=float32, numpy=
 array([[0.7589085 , 0.7807328 , 0.8005964 , ..., 0.71710664, 0.72557086,
         0.74492353],
        [0.        , 0.8034138 , 0.535701  , ..., 1.        , 1.        ,
         1.        ],
        [0.537607  , 0.34066024, 0.3166743 , ..., 0.314

## Get model and set hyperparameters

In [ ]:
%run $MODEL_FILE

In [ ]:
shutil.copy('/content/drive/My Drive/Colab Notebooks/demo-riiid-transformer.ipynb', OUTPUT_FOLDER)

'19-Dec-riiid-2/demo-riiid-transformer.ipynb'

In [ ]:
# Hyperparameters main model
config = {"features": selections, "n_features": len(selections), "vocab_sizes": vocab_sizes, 
          "enc_emb": ENC_EMB, "enc_dense": ENC_DENSE, "dec_emb": DEC_EMB, "dec_dense": DEC_DENSE, 
          "window_size": THR_E, "enc_num_layers": 2, "dec_num_layers": 2, "d_model": 256, 
          "dff": 512, "num_heads": 8, "dropout_rate": 0.1, "pos_encoding": True,
          "padding_values": PADDING_VALUES, "output_types": OUTPUT_TYPES, "padded_shapes": PADDED_SHAPES, 
          "quantile_transformer_et": quantile_transformer_et,  "quantile_transformer_lt": quantile_transformer_lt}

with open(os.path.join(OUTPUT_FOLDER, "config.pickle"), 'wb') as handle:
  pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

logging.info('config: %s', config)

19-12 17:55 root         INFO     config: {'features': ['E', 'r', 'lt'], 'n_features': 3, 'vocab_sizes': [13524, 4, None], 'enc_emb': [0], 'enc_dense': [], 'dec_emb': [1], 'dec_dense': [2], 'window_size': 100, 'enc_num_layers': 2, 'dec_num_layers': 2, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'pos_encoding': True, 'padding_values': (13523, 2, 1.0), 'output_types': (tf.int32, tf.int32, tf.float32), 'padded_shapes': ([100], [101], [100]), 'quantile_transformer_et': [], 'quantile_transformer_lt': QuantileTransformer(random_state=0)}


In [ ]:
transformer = create_model_separate_input(config)
# tf.keras.utils.plot_model(transformer, os.path.join(OUTPUT_FOLDER, "model_plot.png"), show_shapes=True)

## Optimizer

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(config["d_model"])
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999, 
                                      epsilon=1e-9)

## Loss and metrics

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [ ]:
# Loss and metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

accuracy_object = tf.keras.metrics.sparse_categorical_accuracy

train_auc= tf.keras.metrics.AUC()
val_auc= tf.keras.metrics.AUC()

In [ ]:
def loss_function(real, pred): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    loss_ = loss_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss_ *= mask 
    loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask) #loss becomes one value! (from all batches)
    return loss_

In [ ]:
def metric_function(real, pred, auc_object): # batch_size x seq_size x 1 vs  batch_size x seq_size x 3
    # Predict accuracy
    mask = tf.math.logical_not(tf.math.equal(tf.squeeze(real), N_response)) # batch_size x seq_size
    accuracy = accuracy_object(real, pred) # batch_size x seq_size
    mask = tf.cast(mask, dtype=accuracy.dtype)
    
    accuracy *= mask
    accuracy = 100*tf.reduce_sum(accuracy)/tf.reduce_sum(mask)
    
    # A work-around to predict AUC => is it stable?
    pred = tf.nn.softmax(pred)
    pred = pred[:,:,1] # pred that answer is correct
    real = tf.keras.backend.flatten(real)
    pred = tf.keras.backend.flatten(pred)
    
    idxs = tf.math.logical_not(tf.math.equal(real, N_response))
    real = real[idxs]
    pred = pred[idxs]
    auc = auc_object(real, pred)
    return accuracy

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.Mean(name='val_accuracy')

## Training and checkpointing

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [ ]:
checkpoint_path = os.path.join(OUTPUT_FOLDER, "checkpoints/")
ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# https://stackoverflow.com/questions/62919208/how-to-restore-a-specific-checkpoint-in-tensorflow2-to-implement-early-stopping

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [ ]:
@tf.function()
def train_step(tar_real, *inputs):
    with tf.GradientTape() as tape: 
        predictions = transformer(inputs, training=True)
        tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
        
        loss = loss_function(tar_real, predictions)
        accuracy = metric_function(tar_real, predictions, auc_object = train_auc)
    
    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(accuracy)

@tf.function()
def val_step(tar_real, *inputs):
    predictions = transformer(inputs, training=False)
    tar_real = tf.expand_dims(tar_real, -1) # IMPORTANT! DOESN'T WORK WITHOUT IT. ALWAYS GIVES ERROR INCOMPATIBLE SHAPE. E.G. (32,169) vs. (32,169,3)
    
    loss = loss_function(tar_real, predictions)
    accuracy = metric_function(tar_real, predictions, auc_object = val_auc)

    val_loss(loss)
    val_accuracy(accuracy)

In [ ]:
best_auc = 0
for epoch in range(EPOCHS):
    train_dataset = create_train_dataset(N_SELECT_PER_EPOCH, PROBS_TRAIN) # create train for each epoch
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_auc.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_auc.reset_states()

    # Train
    for (batch, (features)) in enumerate(train_dataset):
        E, tar, rest = features[0], features[1], features[2:] 
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        inputs = [E, tar_inp] + list(rest)        
        train_step(tar_real, inputs)
        
        if batch % PRINT_EVERY_N_BATCHES == 0:
            print ('Epoch {} TRAIN Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result(), train_auc.result()))

    logging.info('Epoch TRAIN {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result(),
                                                train_auc.result()
                                                ))
    
    # Validate and save model
    if (epoch + 1) % VAL_EVERY_N_EPOCHS == 0: 
      for (batch, (features)) in enumerate(val_dataset):
          E, tar, rest = features[0], features[1], features[2:] 
          tar_inp = tar[:, :-1]
          tar_real = tar[:, 1:]

          inputs = [E, tar_inp] + list(rest)          
          val_step(tar_real, inputs)

          if batch % PRINT_EVERY_N_BATCHES == 0:
              print ('Epoch {} VAL Batch {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(
                epoch + 1, batch, val_loss.result(), val_accuracy.result(), val_auc.result()))
      
      logging.info('Epoch VAL {} Loss {:.4f} Accuracy {:.4f} AUC {:.4f}'.format(epoch + 1, 
                                          val_loss.result(), 
                                          val_accuracy.result(),
                                          val_auc.result()
                                          ))    
       
      if val_auc.result() > best_auc:
          best_auc = val_auc.result()
          ckpt_save_path = ckpt_manager.save()
          logging.info('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                            ckpt_save_path))
        
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 TRAIN Batch 0 Loss 3.0498 Accuracy 0.1195 AUC 0.4931
Epoch 1 TRAIN Batch 50 Loss 1.5463 Accuracy 32.9710 AUC 0.4977
Epoch 1 TRAIN Batch 100 Loss 1.1247 Accuracy 47.4041 AUC 0.4969
Epoch 1 TRAIN Batch 150 Loss 0.9774 Accuracy 52.5494 AUC 0.4981
Epoch 1 TRAIN Batch 200 Loss 0.9001 Accuracy 55.4431 AUC 0.4998
Epoch 1 TRAIN Batch 250 Loss 0.8523 Accuracy 57.2810 AUC 0.5012
Epoch 1 TRAIN Batch 300 Loss 0.8183 Accuracy 58.7727 AUC 0.5028
Epoch 1 TRAIN Batch 350 Loss 0.7936 Accuracy 59.8625 AUC 0.5040
Epoch 1 TRAIN Batch 400 Loss 0.7748 Accuracy 60.6998 AUC 0.5052
Epoch 1 TRAIN Batch 450 Loss 0.7602 Accuracy 61.3331 AUC 0.5062
Epoch 1 TRAIN Batch 500 Loss 0.7481 Accuracy 61.8890 AUC 0.5070
Epoch 1 TRAIN Batch 550 Loss 0.7382 Accuracy 62.3381 AUC 0.5078
Epoch 1 TRAIN Batch 600 Loss 0.7297 Accuracy 62.7217 AUC 0.5090
Epoch 1 TRAIN Batch 650 Loss 0.7222 Accuracy 63.0712 AUC 0.5102
Epoch 1 TRAIN Batch 700 Loss 0.7160 Accuracy 63.3531 AUC 0.5115
Epoch 1 TRAIN Batch 750 Loss 0.7104 Accuracy

19-12 17:57 root         INFO     Epoch TRAIN 1 Loss 0.7074 Accuracy 63.7398 AUC 0.5139


Time taken for 1 epoch: 100.97339701652527 secs

Epoch 2 TRAIN Batch 0 Loss 0.6372 Accuracy 66.3282 AUC 0.5442
Epoch 2 TRAIN Batch 50 Loss 0.6287 Accuracy 67.3050 AUC 0.5563
Epoch 2 TRAIN Batch 100 Loss 0.6288 Accuracy 67.1686 AUC 0.5616
Epoch 2 TRAIN Batch 150 Loss 0.6283 Accuracy 67.1415 AUC 0.5662
Epoch 2 TRAIN Batch 200 Loss 0.6271 Accuracy 67.1705 AUC 0.5718
Epoch 2 TRAIN Batch 250 Loss 0.6255 Accuracy 67.2586 AUC 0.5774
Epoch 2 TRAIN Batch 300 Loss 0.6253 Accuracy 67.2521 AUC 0.5796
Epoch 2 TRAIN Batch 350 Loss 0.6239 Accuracy 67.3188 AUC 0.5848
Epoch 2 TRAIN Batch 400 Loss 0.6225 Accuracy 67.4112 AUC 0.5887
Epoch 2 TRAIN Batch 450 Loss 0.6215 Accuracy 67.4711 AUC 0.5921
Epoch 2 TRAIN Batch 500 Loss 0.6210 Accuracy 67.4761 AUC 0.5947
Epoch 2 TRAIN Batch 550 Loss 0.6202 Accuracy 67.5217 AUC 0.5970
Epoch 2 TRAIN Batch 600 Loss 0.6196 Accuracy 67.5543 AUC 0.5987
Epoch 2 TRAIN Batch 650 Loss 0.6192 Accuracy 67.5724 AUC 0.6003
Epoch 2 TRAIN Batch 700 Loss 0.6187 Accuracy 67.5937 AUC 0

19-12 17:58 root         INFO     Epoch TRAIN 2 Loss 0.6180 Accuracy 67.6393 AUC 0.6039


Time taken for 1 epoch: 94.63800120353699 secs

Epoch 3 TRAIN Batch 0 Loss 0.6199 Accuracy 66.8235 AUC 0.6332
Epoch 3 TRAIN Batch 50 Loss 0.6125 Accuracy 67.9109 AUC 0.6205
Epoch 3 TRAIN Batch 100 Loss 0.6128 Accuracy 67.9106 AUC 0.6186
Epoch 3 TRAIN Batch 150 Loss 0.6125 Accuracy 67.9170 AUC 0.6201
Epoch 3 TRAIN Batch 200 Loss 0.6127 Accuracy 67.8962 AUC 0.6204
Epoch 3 TRAIN Batch 250 Loss 0.6120 Accuracy 67.9632 AUC 0.6213
Epoch 3 TRAIN Batch 300 Loss 0.6114 Accuracy 68.0178 AUC 0.6226
Epoch 3 TRAIN Batch 350 Loss 0.6101 Accuracy 68.0792 AUC 0.6264
Epoch 3 TRAIN Batch 400 Loss 0.6067 Accuracy 68.2869 AUC 0.6346
Epoch 3 TRAIN Batch 450 Loss 0.6026 Accuracy 68.5340 AUC 0.6447
Epoch 3 TRAIN Batch 500 Loss 0.5984 Accuracy 68.8058 AUC 0.6543
Epoch 3 TRAIN Batch 550 Loss 0.5948 Accuracy 69.0257 AUC 0.6621
Epoch 3 TRAIN Batch 600 Loss 0.5915 Accuracy 69.2367 AUC 0.6687
Epoch 3 TRAIN Batch 650 Loss 0.5885 Accuracy 69.4243 AUC 0.6747
Epoch 3 TRAIN Batch 700 Loss 0.5857 Accuracy 69.6001 AUC 0.

...

Time taken for 1 epoch: 85.14265275001526 secs

Epoch 200 TRAIN Batch 0 Loss 0.5235 Accuracy 73.4271 AUC 0.7748
Epoch 200 TRAIN Batch 50 Loss 0.5089 Accuracy 74.4427 AUC 0.7907
Epoch 200 TRAIN Batch 100 Loss 0.5075 Accuracy 74.5450 AUC 0.7917
Epoch 200 TRAIN Batch 150 Loss 0.5065 Accuracy 74.6170 AUC 0.7926
Epoch 200 TRAIN Batch 200 Loss 0.5065 Accuracy 74.6142 AUC 0.7928
Epoch 200 TRAIN Batch 250 Loss 0.5067 Accuracy 74.5940 AUC 0.7926
Epoch 200 TRAIN Batch 300 Loss 0.5064 Accuracy 74.6122 AUC 0.7927
Epoch 200 TRAIN Batch 350 Loss 0.5062 Accuracy 74.6293 AUC 0.7928
Epoch 200 TRAIN Batch 400 Loss 0.5061 Accuracy 74.6298 AUC 0.7928
Epoch 200 TRAIN Batch 450 Loss 0.5061 Accuracy 74.6299 AUC 0.7928
Epoch 200 TRAIN Batch 500 Loss 0.5062 Accuracy 74.6292 AUC 0.7926
Epoch 200 TRAIN Batch 550 Loss 0.5061 Accuracy 74.6302 AUC 0.7926
Epoch 200 TRAIN Batch 600 Loss 0.5061 Accuracy 74.6325 AUC 0.7926
Epoch 200 TRAIN Batch 650 Loss 0.5062 Accuracy 74.6271 AUC 0.7926
Epoch 200 TRAIN Batch 700 Loss 

19-12 22:57 root         INFO     Epoch TRAIN 200 Loss 0.5062 Accuracy 74.6264 AUC 0.7926


Epoch 200 VAL Batch 0 Loss 0.5134 Accuracy 74.2934 AUC 0.7846
Epoch 200 VAL Batch 50 Loss 0.5128 Accuracy 74.2732 AUC 0.7876
Epoch 200 VAL Batch 100 Loss 0.5120 Accuracy 74.3172 AUC 0.7883
Epoch 200 VAL Batch 150 Loss 0.5119 Accuracy 74.3308 AUC 0.7882
Epoch 200 VAL Batch 200 Loss 0.5120 Accuracy 74.3087 AUC 0.7883
Epoch 200 VAL Batch 250 Loss 0.5117 Accuracy 74.3332 AUC 0.7885
Epoch 200 VAL Batch 300 Loss 0.5114 Accuracy 74.3560 AUC 0.7887
Epoch 200 VAL Batch 350 Loss 0.5115 Accuracy 74.3567 AUC 0.7888
Epoch 200 VAL Batch 400 Loss 0.5112 Accuracy 74.3735 AUC 0.7890
Epoch 200 VAL Batch 450 Loss 0.5109 Accuracy 74.4000 AUC 0.7891
Epoch 200 VAL Batch 500 Loss 0.5109 Accuracy 74.3978 AUC 0.7891
Epoch 200 VAL Batch 550 Loss 0.5110 Accuracy 74.3962 AUC 0.7889
Epoch 200 VAL Batch 600 Loss 0.5107 Accuracy 74.4129 AUC 0.7891
Epoch 200 VAL Batch 650 Loss 0.5108 Accuracy 74.4073 AUC 0.7891
Epoch 200 VAL Batch 700 Loss 0.5108 Accuracy 74.4084 AUC 0.7891
Epoch 200 VAL Batch 750 Loss 0.5108 Accurac

19-12 22:58 root         INFO     Epoch VAL 200 Loss 0.5108 Accuracy 74.4116 AUC 0.7891


Time taken for 1 epoch: 102.58798742294312 secs



In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks(OUTPUT_FOLDER) #lists all downloadable files on server

19-Dec-riiid-2/
  logger.log
  config.pickle
  demo-riiid-transformer.ipynb
19-Dec-riiid-2/checkpoints/
  checkpoint
  ckpt-38.index
  ckpt-38.data-00000-of-00001
19-Dec-riiid-2/seq-dict-100/
  lt.pickle
  r.pickle
  E.pickle

## Add output to Drive

In [ ]:
OUTPUT_DRIVE = "/content/drive/My Drive/kaggle-riiid/subs"

In [ ]:
# https://stackoverflow.com/questions/15034151/copy-directory-contents-into-a-directory-with-python
from distutils.dir_util import copy_tree
copy_tree(OUTPUT_FOLDER, os.path.join(OUTPUT_DRIVE, OUTPUT_FOLDER))

['/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/checkpoints/checkpoint',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/checkpoints/ckpt-38.index',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/checkpoints/ckpt-38.data-00000-of-00001',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/logger.log',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/config.pickle',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/seq-dict-100/lt.pickle',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/seq-dict-100/r.pickle',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/seq-dict-100/E.pickle',
 '/content/drive/My Drive/kaggle-riiid/subs/19-Dec-riiid-2/demo-riiid-transformer.ipynb']

## Add output to Kaggle

In [ ]:
KAGGLE_JSON = "/content/drive/My\ Drive/kaggle-riiid/kaggle.json"
!mkdir -p ~/.kaggle
!cp $KAGGLE_JSON ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets init -p {OUTPUT_FOLDER}

Data package template written to: 19-Dec-riiid-2/dataset-metadata.json


In [ ]:
import json

with open(f'{OUTPUT_FOLDER}/dataset-metadata.json', 'r+') as f:
    data = json.load(f)
    data['title'] = OUTPUT_FOLDER
    data['id'] = f'rafiko1/{OUTPUT_FOLDER}'
    f.seek(0)
    json.dump(data, f, indent=4)
    f.truncate()

!cat {OUTPUT_FOLDER}/dataset-metadata.json

{
    "licenses": [
        {
            "name": "CC0-1.0"
        }
    ],
    "id": "rafiko1/19-Dec-riiid-2",
    "title": "19-Dec-riiid-2"
}

In [ ]:
!kaggle datasets create -p {OUTPUT_FOLDER} -q -r zip 

Your private Dataset is being created. Please check progress at https://www.kaggle.com/rafiko1/19-Dec-riiid-2


In [ ]:
os.listdir(OUTPUT_FOLDER)

['checkpoints',
 'logger.log',
 'dataset-metadata.json',
 'config.pickle',
 'seq-dict-100',
 'demo-riiid-transformer.ipynb']

## Summary

In this tutorial, you learned about positional encoding, multi-head attention, the importance of masking and how to create a transformer.

Try using a different dataset to train the transformer. You can also create the base transformer or transformer XL by changing the hyperparameters above. You can also use the layers defined here to create [BERT](https://arxiv.org/abs/1810.04805) and train state of the art models. Futhermore, you can implement beam search to get better predictions.

# Older code

In [ ]:
# # From https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, df, shuffle=True):
#         'Initialization'
        
#         self.df = df
#         self.users = self.df["user_id"].nunique()
#         self.batch_size = BATCH_SIZE
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return int(np.floor(len(self.list_IDs) / self.batch_size))

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         # Generate indexes of the batch
#         indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

#         # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

#         # Generate data
#         X, y = self.__data_generation(list_IDs_temp)

#         return X, y

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         self.indexes = np.arange(len(self.list_IDs))
#         if self.shuffle == True:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, list_IDs_temp):
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         # Initialization
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         y = np.empty((self.batch_size), dtype=int)

#         # Generate data
#         for i, ID in enumerate(list_IDs_temp):
#             # Store sample
#             X[i,] = np.load('data/' + ID + '.npy')

#             # Store class
#             y[i] = self.labels[ID]

#         return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# def save_dict(seq_dict, filename):
#     with open(os.path.join(OUTPUT_FOLDER, filename), 'wb') as handle:
#       pickle.dump(seq_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Save last N dict to memory
# E_dict = E_lists.apply(lambda x: x[-200:]).to_dict()
# r_dict = r_lists.apply(lambda x: x[-200:]).to_dict()
# et_dict = et_lists.apply(lambda x: x[-200:]).to_dict()

In [ ]:
# save_dict(E_dict, "E.pkl")
# save_dict(r_dict, "r.pkl")
# save_dict(et_dict, "et.pkl")